In [1]:
import os
import numpy as np
import optuna
from skimage import io
from tqdm import tqdm

from chainercv.evaluations import eval_semantic_segmentation

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def get_gt_mask(label_image):
    h, w = label_image.shape[0], label_image.shape[1]
    mask = np.zeros(shape=[h, w], dtype=np.uint8)  # 0: "background"
    mask[(label_image==[0, 0, 255]).sum(axis=2) == 3] = 1      # 1: "car"
    mask[(label_image==[255, 0, 0]).sum(axis=2) == 3] = 2      # 2: "pedestrian"
    mask[(label_image==[255, 255, 0]).sum(axis=2) == 3] = 3 # 3: "signal"
    mask[(label_image==[69, 47, 142]).sum(axis=2) == 3] = 4 # 4: "lane" (road + parking)
    return mask

In [3]:
def  get_pred_mask_array(score_array, weight=None):
    weight = np.ones(shape=[1, 1, 1, 5]) if (weight is None) else np.array([[[weight]]])
    score_weighted = score_array * weight
    mask_array = np.argmax(score_weighted, axis=3)
    return mask_array

In [12]:
label_dir = '../../data/aiedge/seg_train_annotations'
score_dir = '../../results_old/aiedge_s0.5_clahe_nocdist_bw32/model_epoch_93/val_score'

#gt_mask_array = np.empty(shape=[0, 1216, 1936], dtype=np.uint8)
#score_array = np.empty(shape=[0, 1216, 1936, 5], dtype=np.float32)

gt_mask_list = []
score_list = []

score_files = os.listdir(score_dir)
for score_file in tqdm(score_files[:100]):
    
    # Load pred score
    score_path = os.path.join(score_dir, score_file)
    score = np.load(score_path)
    
    # Load gt mask
    basename = os.path.basename(score_file)
    basename, _ = os.path.splitext(basename)
    label_file = '{}.png'.format(basename)
    label_path = os.path.join(label_dir, label_file)
    label = io.imread(label_path)
    gt_mask = get_gt_mask(label)
    
    # Cache
    #gt_mask_array = np.concatenate([gt_mask_array, gt_mask[None,:,:]], axis=0)
    #score_array = np.concatenate([score_array, score[None,:,:,:]], axis=0)
    gt_mask_list.append(gt_mask[None, :, :])
    score_list.append(score[None, :, :, :])

gt_mask_array = np.concatenate(gt_mask_list, axis=0)
score_array = np.concatenate(score_list, axis=0)

print('Done!')

100%|██████████| 100/100 [00:54<00:00,  1.50it/s]


Done!


In [28]:
weight = [1,1,1,1.5,1]
pred_mask_array = get_pred_mask_array(score_array, weight)

In [29]:
ss_eval = eval_semantic_segmentation(pred_mask_array, gt_mask_array)

In [30]:
iou = ss_eval['iou'][1:]
miou = np.nanmean(iou)
print(miou, iou)

0.7239648334418165 [0.84470835 0.63764871 0.52060111 0.89290116]


In [31]:
def compute_miou(class_weight):
    pred_mask_array = get_pred_mask_array(score_array, class_weight)
    ss_eval = eval_semantic_segmentation(pred_mask_array, gt_mask_array)
    iou = ss_eval['iou'][1:]
    miou = np.nanmean(iou)
    return miou

In [32]:
def objective(trial):
    car = 1.0 #trial.suggest_uniform('car', 0, 10)
    pedestrian = 1.0 #trial.suggest_uniform('pedestrian', 0, 10)
    signal = trial.suggest_uniform('signal', 0, 5)
    lane = 1.0 #trial.suggest_uniform('lane', 0, 10)
    
    class_weight = [1.0, car, pedestrian, signal, lane]
    
    miou = compute_miou(class_weight)
    
    return 1.0 - miou

In [33]:
study = optuna.create_study()
study.optimize(objective, n_trials=300)

[I 2019-01-04 08:42:01,436] Finished a trial resulted in value: 0.27854080262371306. Current best value is 0.27854080262371306 with parameters: {'signal': 3.3184275168296153}.
[I 2019-01-04 08:42:24,864] Finished a trial resulted in value: 0.2797138805541899. Current best value is 0.27854080262371306 with parameters: {'signal': 3.3184275168296153}.
[I 2019-01-04 08:42:49,438] Finished a trial resulted in value: 0.2788652436752175. Current best value is 0.27854080262371306 with parameters: {'signal': 3.3184275168296153}.
[I 2019-01-04 08:43:13,841] Finished a trial resulted in value: 0.2805310685599365. Current best value is 0.27854080262371306 with parameters: {'signal': 3.3184275168296153}.
[I 2019-01-04 08:43:37,256] Finished a trial resulted in value: 0.27626120201429205. Current best value is 0.27626120201429205 with parameters: {'signal': 1.3555165184643336}.
[I 2019-01-04 08:43:59,930] Finished a trial resulted in value: 0.2803726995872369. Current best value is 0.276261202014292

[I 2019-01-04 09:00:40,545] Finished a trial resulted in value: 0.2763119342265554. Current best value is 0.27601107361765154 with parameters: {'signal': 1.6182489436132916}.
[I 2019-01-04 09:01:06,682] Finished a trial resulted in value: 0.2810293264956847. Current best value is 0.27601107361765154 with parameters: {'signal': 1.6182489436132916}.
[I 2019-01-04 09:01:31,064] Finished a trial resulted in value: 0.29148229819127636. Current best value is 0.27601107361765154 with parameters: {'signal': 1.6182489436132916}.
[I 2019-01-04 09:01:54,323] Finished a trial resulted in value: 0.2760255536316767. Current best value is 0.27601107361765154 with parameters: {'signal': 1.6182489436132916}.
[I 2019-01-04 09:02:20,679] Finished a trial resulted in value: 0.27894040656050045. Current best value is 0.27601107361765154 with parameters: {'signal': 1.6182489436132916}.
[I 2019-01-04 09:02:47,334] Finished a trial resulted in value: 0.2760882774944664. Current best value is 0.276011073617651

[I 2019-01-04 09:19:33,027] Finished a trial resulted in value: 0.27726453084435554. Current best value is 0.2760097002648636 with parameters: {'signal': 1.629231214214867}.
[I 2019-01-04 09:19:58,188] Finished a trial resulted in value: 0.2777799215022335. Current best value is 0.2760097002648636 with parameters: {'signal': 1.629231214214867}.
[I 2019-01-04 09:20:22,980] Finished a trial resulted in value: 0.2768761066579202. Current best value is 0.2760097002648636 with parameters: {'signal': 1.629231214214867}.
[I 2019-01-04 09:20:48,101] Finished a trial resulted in value: 0.27850024887350844. Current best value is 0.2760097002648636 with parameters: {'signal': 1.629231214214867}.
[I 2019-01-04 09:21:13,091] Finished a trial resulted in value: 0.2761531488023412. Current best value is 0.2760097002648636 with parameters: {'signal': 1.629231214214867}.
[I 2019-01-04 09:21:37,779] Finished a trial resulted in value: 0.2760630022858761. Current best value is 0.2760097002648636 with par

[I 2019-01-04 09:38:37,424] Finished a trial resulted in value: 0.2794777632310077. Current best value is 0.27600901066170525 with parameters: {'signal': 1.534534119602365}.
[I 2019-01-04 09:38:59,985] Finished a trial resulted in value: 0.27660452629539467. Current best value is 0.27600901066170525 with parameters: {'signal': 1.534534119602365}.
[I 2019-01-04 09:39:22,669] Finished a trial resulted in value: 0.28069628554163284. Current best value is 0.27600901066170525 with parameters: {'signal': 1.534534119602365}.
[I 2019-01-04 09:39:43,462] Finished a trial resulted in value: 0.2787159571032085. Current best value is 0.27600901066170525 with parameters: {'signal': 1.534534119602365}.
[I 2019-01-04 09:40:02,086] Finished a trial resulted in value: 0.27613637412609315. Current best value is 0.27600901066170525 with parameters: {'signal': 1.534534119602365}.
[I 2019-01-04 09:40:22,736] Finished a trial resulted in value: 0.2763180968289234. Current best value is 0.27600901066170525 w

[I 2019-01-04 09:57:08,593] Finished a trial resulted in value: 0.2760178400794353. Current best value is 0.275992014264536 with parameters: {'signal': 1.5856692838423545}.
[I 2019-01-04 09:57:32,390] Finished a trial resulted in value: 0.2761917281798445. Current best value is 0.275992014264536 with parameters: {'signal': 1.5856692838423545}.
[I 2019-01-04 09:57:54,645] Finished a trial resulted in value: 0.3354229059887557. Current best value is 0.275992014264536 with parameters: {'signal': 1.5856692838423545}.
[I 2019-01-04 09:58:17,500] Finished a trial resulted in value: 0.2768824104642502. Current best value is 0.275992014264536 with parameters: {'signal': 1.5856692838423545}.
[I 2019-01-04 09:58:40,462] Finished a trial resulted in value: 0.27616620537934433. Current best value is 0.275992014264536 with parameters: {'signal': 1.5856692838423545}.
[I 2019-01-04 09:59:03,093] Finished a trial resulted in value: 0.2764206418991535. Current best value is 0.275992014264536 with param

[I 2019-01-04 10:15:35,639] Finished a trial resulted in value: 0.27806016858495086. Current best value is 0.275992014264536 with parameters: {'signal': 1.5856692838423545}.
[I 2019-01-04 10:16:01,480] Finished a trial resulted in value: 0.2761317512292043. Current best value is 0.275992014264536 with parameters: {'signal': 1.5856692838423545}.
[I 2019-01-04 10:16:26,749] Finished a trial resulted in value: 0.277291831520482. Current best value is 0.275992014264536 with parameters: {'signal': 1.5856692838423545}.
[I 2019-01-04 10:16:53,760] Finished a trial resulted in value: 0.27601842210187666. Current best value is 0.275992014264536 with parameters: {'signal': 1.5856692838423545}.
[I 2019-01-04 10:17:16,791] Finished a trial resulted in value: 0.27630623777804697. Current best value is 0.275992014264536 with parameters: {'signal': 1.5856692838423545}.
[I 2019-01-04 10:17:39,297] Finished a trial resulted in value: 0.2776039280439311. Current best value is 0.275992014264536 with para

[I 2019-01-04 10:34:21,327] Finished a trial resulted in value: 0.2775490463980108. Current best value is 0.275992014264536 with parameters: {'signal': 1.5856692838423545}.
[I 2019-01-04 10:34:44,513] Finished a trial resulted in value: 0.28560965848446274. Current best value is 0.275992014264536 with parameters: {'signal': 1.5856692838423545}.
[I 2019-01-04 10:35:06,227] Finished a trial resulted in value: 0.27637336453486405. Current best value is 0.275992014264536 with parameters: {'signal': 1.5856692838423545}.
[I 2019-01-04 10:35:30,278] Finished a trial resulted in value: 0.27646503090909946. Current best value is 0.275992014264536 with parameters: {'signal': 1.5856692838423545}.
[I 2019-01-04 10:35:56,393] Finished a trial resulted in value: 0.27605051906072586. Current best value is 0.275992014264536 with parameters: {'signal': 1.5856692838423545}.
[I 2019-01-04 10:36:21,104] Finished a trial resulted in value: 0.27681261805748447. Current best value is 0.275992014264536 with p

KeyboardInterrupt: 